In [1]:
from imports import *
from utils import *
from model import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading

dataset = 'Google Speech Command (GSC)'

data_path = 'raw/speech_commands'
labels = ['yes', 'no', 'stop','on','off']

train_dataset, test_dataset = load_speech_command_datasets(data_path, labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Class 'yes' has 2377 audio files.
Class 'no' has 2375 audio files.
Class 'stop' has 2380 audio files.
Class 'on' has 2367 audio files.
Class 'off' has 2357 audio files.


In [5]:
# Model Definition

# Define the number of classes for the classification task
num_classes = 5

# Initialize the model with the given configuration and number of classes, and move it to the specified device
model = NET_4k_GSC(num_classes).to(device)

total_weights = count_weights(model)
print(f"The network has {total_weights} weights that need to be deployed.")

model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
model = torch.quantization.prepare_qat(model, inplace=True)
model = torch.quantization.convert(model, inplace=True)

The network has 465 weights that need to be deployed.


In [6]:
# Load Pretrained Model

# Specify the filename of the saved model
model_filename = f"checkpoint/GSC-465-94.10_model.t7"

# Load the checkpoint from the file, mapping the model to the specified device
checkpoint = torch.load(model_filename, map_location=device)

# Load the model state dictionary from the checkpoint
model.load_state_dict(checkpoint['model'])

print(model) # The model has been quantized to int8

NET_4k_GSC(
  (quant): Quantize(scale=tensor([0.0079]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (conv1_depthwise): QuantizedConv2d(1, 1, kernel_size=(5, 5), stride=(1, 1), scale=0.11747653037309647, zero_point=98, padding=(1, 1), dilation=(2, 2))
  (conv1_pointwise): QuantizedConv2d(1, 10, kernel_size=(1, 1), stride=(1, 1), scale=0.5115418434143066, zero_point=63)
  (bn1): QuantizedBatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): QuantizedHardswish()
  (conv2_depthwise): QuantizedConv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), scale=0.27415165305137634, zero_point=72, padding=(1, 1), dilation=(2, 2), groups=10)
  (conv2_pointwise): QuantizedConv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), scale=0.9521254897117615, zero_point=64)
  (bn2): QuantizedBatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation2): QuantizedHardswish()
  (conv3_depthwise): QuantizedConv2d(10, 1

In [4]:
# Test the model on the test dataset and store the accuracy
model.eval()
test_accuracy = test_model(model, test_loader)

# Extract the int8 weights from all quantized layers in the model and save them to a list
quantized_weights_list = print_quantized_weights(model)

print(f"The test accuracy after quantization is: {test_accuracy:.2f}%")

Extracting weights from layer: conv1_depthwise
Weight Shape: (25,), Int8 Weights:
[   6    6   -9  -42   21   23  -17 -128  -74   32   43  -60  -75   -5
   23    5  -57  -44  -24    1  -11   56   29    9   14]

Extracting weights from layer: conv1_pointwise
Weight Shape: (10,), Int8 Weights:
[-128 -128  127  118 -128 -128  127 -128  127  127]

Extracting weights from layer: conv2_depthwise
Weight Shape: (90,), Int8 Weights:
[  -7   36  127    1    4  -39  -33  -12   13   53 -128   25   41   -8
  -42 -109  111   34   20   54   -3   53  -36  -30   -9   14  127  -15
   16   60  103   35   82  -26 -128  -36  -28  -66   61   27  127  -99
  -40  -52   30  -28  -43   91  127  -51  -30   50   47  -87   42 -128
    1   -2   71  -30    3   45   -6   33  -17    0  -25   71 -128   30
   37  -46   78 -124  -79   23  -19  113  -59 -126   48  -55 -128  -64
   23  -25   78   46  -36   44]

Extracting weights from layer: conv2_pointwise
Weight Shape: (100,), Int8 Weights:
[ -26 -128   16  -95  -72   71